<a href="https://colab.research.google.com/github/ThiagoJFreitas/Rathena_tools/blob/main/Rathena_bluebox_To_YML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import yaml
import os

# Função para carregar todos os arquivos YAML que começam com 'item_db' e criar um dicionário {ID: AegisName}
def carregar_todos_item_db(pasta):
    id_para_aegisname = {}

    for arquivo in os.listdir(pasta):
        if arquivo.startswith('item_db') and arquivo.endswith('.yml'):
            caminho_arquivo = os.path.join(pasta, arquivo)
            with open(caminho_arquivo, 'r') as f:
                dados = yaml.safe_load(f)

            for item in dados['Body']:
                id_para_aegisname[int(item['Id'])] = item['AegisName']

    return id_para_aegisname

# Função para processar o arquivo de entrada TXT e gerar a estrutura YAML
def processar_arquivo_txt(caminho_txt, id_para_aegisname, caminho_saida):
    estrutura_yaml = []
    group_items = {}

    with open(caminho_txt, 'r') as f:
        linhas = f.readlines()

    index = 0  # Inicializa o índice crescente

    for linha in linhas:
        # Ignorar comentários ou linhas em branco
        if not linha.strip() or linha.strip().startswith('//'):
            continue

        try:
            # Separar os elementos da linha
            partes = linha.split(',')
            group = partes[0].strip()
            item_id = int(partes[1].strip())
            rate = int(partes[2].split('//')[0].strip())

            # Obter o AegisName do item_db usando o ID
            item_name = id_para_aegisname.get(item_id)

            if not item_name:
                print(f"Aviso: ID {item_id} não encontrado no banco de dados. Ignorando linha.")
                continue

            # Agrupar itens por Group
            group_upper = group.upper()
            if group_upper not in group_items:
                group_items[group_upper] = []

            group_items[group_upper].append({
                'Index': index,
                'Item': item_name,
                'Rate': rate
            })

            index += 1  # Incrementa o índice para a próxima entrada

        except (IndexError, ValueError) as e:
            print(f"Erro ao processar a linha: {linha.strip()} - {e}")
            continue

    # Construir a estrutura final em YAML
    for group, items in group_items.items():
        estrutura_yaml.append({
            'Group': group,
            'SubGroups': [
                {
                    'SubGroup': 1,
                    'Algorithm': 'Random',
                    'List': items
                }
            ]
        })

    # Escrever o arquivo de saída em YAML
    with open(caminho_saida, 'w') as f:
        yaml.dump(estrutura_yaml, f, sort_keys=False)

# Caminhos dos arquivos de entrada e saída
pasta_item_db = '.'  # Diretório atual
caminho_txt = 'item_bluebox.txt'
caminho_saida = 'item_group_db.yml'

# Carregar o banco de dados de itens de todos os arquivos que começam com 'item_db'
id_para_aegisname = carregar_todos_item_db(pasta_item_db)

# Processar o arquivo TXT e gerar o arquivo de saída YAML
processar_arquivo_txt(caminho_txt, id_para_aegisname, caminho_saida)

print(f"Arquivo YAML gerado com sucesso em {caminho_saida}")


Arquivo YAML gerado com sucesso em item_group_db.yml
